Saco promedio diario mensual (promedio de cada dia cada mes)

Hago un  ranking de las radiaciones.


3.Tengo dos series de datos y sale 1.


El calculo de la efectividad del método sería ideal hacerlo para diferenmtes años y difrentes periodos (3, 6 y 12 meses)